In [33]:
# Starting to explore stability of commodities using shipping data in the BKB dataset
# 24.07.2024
# vera.provatorova@dh.huc.knaw.nl

## Steps:
* [Read the data](#read-data)
* [Convert shipping quantity values to decimal](#convert-values)
* [Aggregate the data](#aggregate)
* [Connect the commodities with GLOBALISE thesaurus](#globalise)
* [Calculate variation coefficients](#calculate-variation)

### Read the data <a class="anchor" id="read-data"></a>

In [34]:
DATA_DIR = 'data/' # change the path when running to point to your data directory

In [35]:
# First step: read the BKB data
import glob
import pandas as pd
from os.path import splitext, basename

def get_name(path):
    '''
    in: path/to/data/bgb_cargo.tsv
    out: cargo
    '''
    root, ext = splitext(path) # 'path/to/data/bgb_cargo.tsv' -> 'path/to/data/bgb_cargo', '.tsv'
    return basename(root).split('_')[-1] # 'path/to/data/bgb_cargo' -> 'bgb_cargo' -> 'cargo'

bkb_data = {get_name(path): pd.read_csv(path,sep='\t',low_memory=False) 
           for path in glob.glob(DATA_DIR+'bkb_cargo_logs/*')} # we assume the sheets are stored separately as .tsv files

In [36]:
# Display the data
from IPython.display import display
for key, df in bkb_data.items():
    print(key+':')
    display(df.head(3))

ship:


,id,naam,added_when,added_by,timestamp
0,3067,Vrouwe Agatha,5/21/08 15:37,mjo,5/21/08 15:37
1,3076,Faam,5/22/08 11:28,mjo,5/22/08 11:28
2,3065,Dordrecht,5/21/08 15:07,mjo,5/21/08 15:07


source:


,id,naam,added_when,added_by,timestamp
0,105,10758.0,5/6/08 11:47,admin,4/9/08 16:55
1,104,10757.0,5/6/08 11:47,admin,4/9/08 16:55
2,103,10756.0,5/6/08 11:47,admin,4/9/08 16:55


relVoyageShip:


...

,url,voyId,voyBookingDay,voyBookingMonth,voyBookingYear,voyDeparturePlaceId,voyDepartureDay,voyDepartureMonth,voyDepartureYear,voyArrivalPlaceId,...,timestamp,voySourceId,voynumber,voyImage,voyRemarksForEndUser,voyDepartureRegioId,voyArrivalRegioId,voyFolioNummer,all_fields,first_ship_name
0,https://bgb.huygens.knaw.nl/bgb/voyage/1,99351,NaN,NaN,1790,934.0,NaN,NaN,NaN,861.0,...,2013-09-10 14:30:24,147.0,1,NaN,NaN,3185,3129,3,1 Batavia Batavia Amsterdam Republiek 1789 1...,Juffrouw Johanna
1,https://bgb.huygens.knaw.nl/bgb/voyage/2,99352,NaN,NaN,1790,934.0,NaN,NaN,NaN,861.0,...,2013-09-10 14:30:24,147.0,2,NaN,NaN,3185,3129,3,2 Batavia Batavia Amsterdam Republiek 1789 1...,Draak
2,https://bgb.huygens.knaw.nl/bgb/voyage/3,99353,NaN,NaN,1790,934.0,NaN,NaN,NaN,861.0,...,2013-09-10 14:30:24,147.0,3,NaN,NaN,3185,3129,3,3 Batavia Batavia Amsterdam Republiek 1790 1...,Doggersbank


cargo:


,carId,carVoyageId,carProductId,carSpecificationId,carUnit,carQuantity,carQuantityNumeric,carValue,carValueGuldens,carValueStuivers,...,carValueLicht,carValueLichtGuldens,carValueLichtStuivers,carValueLichtPenningen,carRemarks,carOrder,changed_when,changed_by,timestamp,all_fields
0,645880,99353,1290.0,NaN,NaN,NaN,NaN,"1.623,30",1623.0,3.0,...,NaN,NaN,NaN,NaN,NaN,9.0,6/25/08 12:42,jsc,8/5/13 10:50,samen
1,645881,99353,1230.0,848.0,88.0,4,4,"371,8",371.0,8.0,...,NaN,NaN,NaN,NaN,NaN,10.0,5/21/08 14:41,mjo,8/5/13 10:50,"zakhorloge zilveren, voor Japan pees"
2,645877,99353,1133.0,16295.0,88.0,4,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.0,2/25/13 10:09,DorineS,8/5/13 10:50,"moir√© gouden, voor Japan pees"


specification:


,id,naam,added_when,added_by,timestamp
0,652,tot inktkokers,5/6/08 11:55,Admin,3/13/13 15:27
1,15877,Constantia rood,2/6/13 16:30,DorineS,2/6/13 16:30
2,654,arduinen,5/6/08 11:55,Admin,5/6/08 11:55


### Convert shipping quantities to decimal <a class="anchor" id="convert-values"></a>

In [37]:
df_cargo = bkb_data['cargo'].dropna(subset=['carQuantityNumeric','carProductId','carVoyageId'])
df_product = bkb_data['product'].dropna(subset=['id'])
df_voyage = bkb_data['voyage'].dropna(subset=['voyId','voyDepartureYear'])

In [38]:
def quantity_to_decimal(q, qnum): # Cleaning up the messy BKB quantities data
    if q == '4.501,11/12': # One case like this, fixing manually
        return pd.to_numeric(4501 + 11/12)
    if qnum.count('.') < 2:  # already a numb  er
        return pd.to_numeric(qnum)
    if q.find(' ') == -1:
        try:
            return pd.to_numeric(qnum)
        except:
            print('Failed to convert qnum, using q instead: ',q, qnum)
            return pd.to_numeric(q.replace('.',''))
        
    q = q.replace('  ',' ') # removing extra spaces
    q = q.replace('/ ','/')
    q = q.replace(' /','/')
        
    try:
        num, frac = q.split(' ') # eg. 8 1/4
    except:
        print(q, qnum)
        num, frac = q.split(' ')
    try:
        frac = frac.replace(' ','')
        a, b = frac.split('/')
    except:
        print('Fraction problem with: ',q, qnum)
        a, b = frac.split('.') # data entry error, confused . and /
    return pd.to_numeric(num.replace('.','').replace(',','')) + pd.to_numeric(a)/pd.to_numeric(b)
    

In [39]:
import numpy as np

df_product['id'] = df_product['id'].astype(int)
df_voyage['voyId'] = df_voyage['voyId'].astype(str)
df_voyage['voyDepartureYear'] = df_voyage['voyDepartureYear'].astype(int)
df_cargo['carQuantityNumeric'] = df_cargo.apply(lambda row: quantity_to_decimal(row['carQuantity'],
                                                                               row['carQuantityNumeric']),
                                                axis=1)

/var/folders/w6/rvcq1vxj6p58nmp30brgs8dm0000gn/T/ipykernel_17088/649704041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_voyage['voyId'] = df_voyage['voyId'].astype(str)
/var/folders/w6/rvcq1vxj6p58nmp30brgs8dm0000gn/T/ipykernel_17088/649704041.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_voyage['voyDepartureYear'] = df_voyage['voyDepartureYear'].astype(int)


Failed to convert qnum, using q instead:  8770 8.769.231
Failed to convert qnum, using q instead:  60.905 60.905.375
Fraction problem with:  46.126 1.6 46.126.166
Failed to convert qnum, using q instead:  33761 33.761.003
Failed to convert qnum, using q instead:  30660 30.659.997


/var/folders/w6/rvcq1vxj6p58nmp30brgs8dm0000gn/T/ipykernel_17088/649704041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cargo['carQuantityNumeric'] = df_cargo.apply(lambda row: quantity_to_decimal(row['carQuantity'],


### Aggregate the data <a class="anchor" id="aggregate"></a>

In [40]:
# Merge cargo with product and voyage data
df_merged = df_cargo.merge(df_product, left_on='carProductId', right_on='id', how='left')\
                    .merge(df_voyage, left_on='carVoyageId', right_on='voyId', how='left').dropna(subset=['voyDepartureYear'])

df_merged.head()
# Extract year and month for aggregation
# df_merged['voyYearMonth'] = pd.to_datetime(df_merged[['voyBookingYear', 'voyBookingMonth']].astype(str).agg('-'.join, axis=1))



,carId,carVoyageId,carProductId,carSpecificationId,carUnit,carQuantity,carQuantityNumeric,carValue,carValueGuldens,carValueStuivers,...,timestamp,voySourceId,voynumber,voyImage,voyRemarksForEndUser,voyDepartureRegioId,voyArrivalRegioId,voyFolioNummer,all_fields_y,first_ship_name
176,646292,99379,1034.0,731.0,69.0,1.078,1078.0,"533,12",533.0,12.0,...,2013-09-10 14:30:24,147.0,27.0,NaN,NaN,3129.0,3185.0,65,27 Amsterdam Republiek Batavia Batavia 1789 ...,Valk
177,646293,99379,1128.0,NaN,69.0,141,141.0,"176,8",176.0,8.0,...,2013-09-10 14:30:24,147.0,27.0,NaN,NaN,3129.0,3185.0,65,27 Amsterdam Republiek Batavia Batavia 1789 ...,Valk
178,646294,99379,1140.0,NaN,69.0,4.301,4301.0,"709,13",709.0,13.0,...,2013-09-10 14:30:24,147.0,27.0,NaN,NaN,3129.0,3185.0,65,27 Amsterdam Republiek Batavia Batavia 1789 ...,Valk
179,646295,99379,1156.0,852.0,69.0,61.984,61984.0,"7.611,14",7611.0,14.0,...,2013-09-10 14:30:24,147.0,27.0,NaN,NaN,3129.0,3185.0,65,27 Amsterdam Republiek Batavia Batavia 1789 ...,Valk
180,646296,99379,1156.0,838.0,69.0,6.250,6250.0,"1413,9",1413.0,9.0,...,2013-09-10 14:30:24,147.0,27.0,NaN,NaN,3129.0,3185.0,65,27 Amsterdam Republiek Batavia Batavia 1789 ...,Valk


In [41]:
set(df_merged['voyDepartureYear'].tolist())

{1700.0,
 1701.0,
 1702.0,
 1703.0,
 1704.0,
 1706.0,
 1707.0,
 1708.0,
 1709.0,
 1710.0,
 1711.0,
 1712.0,
 1713.0,
 1714.0,
 1715.0,
 1721.0,
 1722.0,
 1723.0,
 1724.0,
 1725.0,
 1726.0,
 1727.0,
 1728.0,
 1729.0,
 1730.0,
 1731.0,
 1732.0,
 1734.0,
 1735.0,
 1736.0,
 1737.0,
 1738.0,
 1739.0,
 1740.0,
 1741.0,
 1742.0,
 1743.0,
 1750.0,
 1751.0,
 1752.0,
 1753.0,
 1754.0,
 1755.0,
 1756.0,
 1757.0,
 1758.0,
 1759.0,
 1760.0,
 1761.0,
 1762.0,
 1763.0,
 1764.0,
 1765.0,
 1766.0,
 1767.0,
 1768.0,
 1769.0,
 1771.0,
 1772.0,
 1773.0,
 1774.0,
 1775.0,
 1776.0,
 1777.0,
 1778.0,
 1779.0,
 1780.0,
 1781.0,
 1782.0,
 1783.0,
 1784.0,
 1785.0,
 1786.0,
 1787.0,
 1788.0,
 1789.0,
 1790.0,
 1800.0,
 1801.0}

In [42]:
df_merged['decade'] = df_merged['voyDepartureYear'].map(lambda x: str((x //10)*10)+'s')


In [77]:
df_merged_clean = df_merged[['naam','carQuantityNumeric','carVoyageId',
                            'voyDepartureYear','decade']].drop_duplicates()

df_merged_clean['appearance_count'] = df_merged_clean.groupby('naam')['naam'].transform('size')
df_merged_clean=df_merged_clean.sort_values(by='naam')
df_merged_clean

,naam,carQuantityNumeric,carVoyageId,voyDepartureYear,decade,appearance_count
114759,AB-boek,30.0,113711,1711.0,1710.0s,3
174993,AB-boek,50.0,117173,1726.0,1720.0s,3
145223,AB-boek,20.0,115463,1713.0,1710.0s,3
107759,AB-bord,25.0,113412,1724.0,1720.0s,6
111417,AB-bord,10.0,113553,1724.0,1720.0s,6
...,...,...,...,...,...,...
17109,zwavelaarde,3671.0,102600,1776.0,1770.0s,13
101999,zwavelaarde,200.0,113197,1703.0,1700.0s,13
17113,zwavelaarde,6329.0,102600,1776.0,1770.0s,13
144412,zwavelaarde,1013.0,115425,1713.0,1710.0s,13


### Connect the commodities with GLOBALISE thesaurus <a class="anchor" id="globalise"></a>

In [144]:
# Connecting with the GLOBALISE commodities
import json

# First, get a list of all labels of commodities to filter our data
commodities_raw = json.load(open('data/commoditiesV1.json','r')) # Data available at: https://globalise-data.diginfra.net/sparql
commodities_labels_list_flat = sum([[item['prefLabelNL']['value']]+[label for label in item['altLabelsNL']['value'].split('; ') if label]
                                     for item in commodities_raw['results']['bindings']],
                                    [])
commodities_labels_set = set([label.lower() for label in commodities_labels_list_flat])

# Next, get categories per commodity
parent_uri = {item['concept']['value']:
             item['concept_broader']['value']
             for item in commodities_raw['results']['bindings']
             if 'concept_broader' in item}

name_by_uri = {item['concept']['value']:
             item['prefLabelNL']['value']
             for item in commodities_raw['results']['bindings']}

uri_by_name = {label.lower():
             item['concept']['value']
               for item in commodities_raw['results']['bindings']
              for label in [item['prefLabelNL']['value']]+[lbl for lbl in item['altLabelsNL']['value'].split('; ') if lbl]
             }

def get_second_broadest_label(url='', cur_label=''):
    '''
    Recursively looking for the second-broadest label (category) in the thesaurus
    '''
    if not url:
        url = uri_by_name[cur_label]
    if url not in parent_uri: # can't go any higher
        return cur_label
    new_url = parent_uri[url]
    new_label = name_by_uri[new_url] if new_url in name_by_uri else name_by_uri[url]
    return get_second_broadest_label(new_url, new_label)

def get_category_by_name(label):
    return get_second_broadest_label(url='',cur_label=label)

In [ ]:
# todo: find less broad categories. maybe find a whole chain of parent labels and then extract what we need

In [145]:
get_category_by_name('opium')

'Dranken en Tabak'

In [143]:
# # Temporary filtering, update later

# # Summing up appearance counts
# appearance_sums = df_merged_clean.groupby('naam')['appearance_count'].transform('sum')

# # Filtering the dataframe
# filtered_df = df_merged_clean[appearance_sums > 99]

# # Display the filtered dataframe
# filtered_df

In [146]:
filtered_df = df_merged_clean[df_merged_clean['naam'].isin(commodities_labels_set)]
filtered_df['category'] = filtered_df['naam'].map(get_category_by_name)
filtered_df

/var/folders/w6/rvcq1vxj6p58nmp30brgs8dm0000gn/T/ipykernel_17088/2198938650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['category'] = filtered_df['naam'].map(get_category_by_name)


,naam,carQuantityNumeric,carVoyageId,voyDepartureYear,decade,appearance_count,category
130095,aam,12.0,114638,1714.0,1710.0s,61,"Verwerkte goederen, hoofdzakelijk ingedeeld na..."
129134,aam,12.0,114607,1714.0,1710.0s,61,"Verwerkte goederen, hoofdzakelijk ingedeeld na..."
128738,aam,12.0,114598,1714.0,1710.0s,61,"Verwerkte goederen, hoofdzakelijk ingedeeld na..."
130757,aam,13.0,114661,1714.0,1710.0s,61,"Verwerkte goederen, hoofdzakelijk ingedeeld na..."
121901,aam,9.0,114109,1703.0,1700.0s,61,"Verwerkte goederen, hoofdzakelijk ingedeeld na..."
...,...,...,...,...,...,...,...
17109,zwavelaarde,3671.0,102600,1776.0,1770.0s,13,"Ruwe materialen, oneetbaar, behalve brandstoffen"
101999,zwavelaarde,200.0,113197,1703.0,1700.0s,13,"Ruwe materialen, oneetbaar, behalve brandstoffen"
17113,zwavelaarde,6329.0,102600,1776.0,1770.0s,13,"Ruwe materialen, oneetbaar, behalve brandstoffen"
144412,zwavelaarde,1013.0,115425,1713.0,1710.0s,13,"Ruwe materialen, oneetbaar, behalve brandstoffen"


In [147]:
set(filtered_df['category'].tolist())

{'Chemicaliën en verwante producten, n.e.g.',
 'Dierlijke en plantaardige oliën, vetten en wassen',
 'Dranken en Tabak',
 'Machines en transportmiddelen',
 'Minerale brandstoffen, smeermiddelen en gerelateerd materieel',
 'NOT YET CLASSIFIED',
 'Niet elders geclasificeerde goederen en transacties',
 'Personen behandeld als goederen',
 'Ruwe materialen, oneetbaar, behalve brandstoffen',
 'Vergoedingen en geldbedragen met een specifiek doel',
 'Verwerkte goederen, hoofdzakelijk ingedeeld naar gebruik',
 'Verwerkte goederen, hoofdzakelijk ingedeeld naar materiaal',
 'Voedsel en levende dieren'}

### Calculate variation coefficients <a class="anchor" id="calculate-variation"></a>

In [150]:
# Group by product and time period to get quantities by decade

df_grouped = filtered_df.groupby(['naam', 'category', 'decade']).agg({'carQuantityNumeric': ['mean','std']}).reset_index()

print(df_grouped)
# df_grouped = df_grouped.groupby(['naam', 'voyDepartureYear'])['carQuantityNumeric'].sum().reset_index()

# Flatten the column names
df_grouped.columns = ['commodity','category', 'decade', 'mean_quantity', 'std_dev_quantity']

# Fill NaNs in std_dev_quantity with 0
df_grouped['std_dev_quantity'] = df_grouped['std_dev_quantity'].fillna(0)

# Calculate Coefficient of Variation (CV)
df_grouped['cv_quantity'] = df_grouped['std_dev_quantity'] / df_grouped['mean_quantity']
df_grouped

             naam                                           category   decade  \
                                                                                
0             aam  Verwerkte goederen, hoofdzakelijk ingedeeld na...  1700.0s   
1             aam  Verwerkte goederen, hoofdzakelijk ingedeeld na...  1710.0s   
2             aam  Verwerkte goederen, hoofdzakelijk ingedeeld na...  1720.0s   
3             aam  Verwerkte goederen, hoofdzakelijk ingedeeld na...  1730.0s   
4             aam  Verwerkte goederen, hoofdzakelijk ingedeeld na...  1750.0s   
...           ...                                                ...      ...   
6764  zwavelaarde   Ruwe materialen, oneetbaar, behalve brandstoffen  1730.0s   
6765  zwavelaarde   Ruwe materialen, oneetbaar, behalve brandstoffen  1750.0s   
6766  zwavelaarde   Ruwe materialen, oneetbaar, behalve brandstoffen  1760.0s   
6767  zwavelaarde   Ruwe materialen, oneetbaar, behalve brandstoffen  1770.0s   
6768  zwavelbloem           

,commodity,category,decade,mean_quantity,std_dev_quantity,cv_quantity
0,aam,"Verwerkte goederen, hoofdzakelijk ingedeeld na...",1700.0s,10.025000,12.187650,1.215726
1,aam,"Verwerkte goederen, hoofdzakelijk ingedeeld na...",1710.0s,12.941176,10.573969,0.817079
2,aam,"Verwerkte goederen, hoofdzakelijk ingedeeld na...",1720.0s,29.058824,38.145233,1.312690
3,aam,"Verwerkte goederen, hoofdzakelijk ingedeeld na...",1730.0s,11.000000,0.000000,0.000000
4,aam,"Verwerkte goederen, hoofdzakelijk ingedeeld na...",1750.0s,12.500000,3.535534,0.282843
...,...,...,...,...,...,...
6764,zwavelaarde,"Ruwe materialen, oneetbaar, behalve brandstoffen",1730.0s,7450.000000,7141.778490,0.958628
6765,zwavelaarde,"Ruwe materialen, oneetbaar, behalve brandstoffen",1750.0s,15256.500000,7433.813591,0.487256
6766,zwavelaarde,"Ruwe materialen, oneetbaar, behalve brandstoffen",1760.0s,186650.500000,249821.532900,1.338446
6767,zwavelaarde,"Ruwe materialen, oneetbaar, behalve brandstoffen",1770.0s,6666.666667,3177.982746,0.476697


In [122]:
'gewicht' in commodities_labels_set

True

In [118]:
df_volatility = df_grouped.groupby('commodity')['cv_quantity'].mean().reset_index()


In [119]:
df_volatility = df_volatility.sort_values('cv_quantity').reset_index().drop('index',axis=1)

In [123]:
df_volatility.tail(30)[::-1]

,commodity,cv_quantity
800,tarwe,4.982490
799,rijst,4.741646
798,zeep,4.614860
797,arak,3.001243
796,laken,2.672935
795,wijn,2.576366
794,zout,2.495702
793,koper,2.342490
792,sekwijn,2.268943
791,teer,2.213844


In [124]:
df_volatility[df_volatility['commodity']=='opium']

,commodity,cv_quantity
755,opium,1.544927


In [125]:
df_volatility[df_volatility['commodity']=='thee']

,commodity,cv_quantity
702,thee,1.242368


In [126]:
df_volatility[df_volatility['cv_quantity']>0].head(30)

,commodity,cv_quantity
2,bergcinnaber,0.036262
3,antimonium,0.068182
4,lheimenias,0.087040
5,juchtleer,0.095740
6,roos (bloem),0.098974
7,hevel,0.101105
8,djarak-olie,0.103750
9,bamboe,0.124252
10,zand,0.133333
11,stempel,0.144016
